### This code calculates the distance between each home in our dataset and a list of several hunder popular attractions found within King County.  The output is used to determine the count of attractions within 7 miles of each house as a potential indicator of a price of a home.

In [1]:
import pandas as pd
import numpy as np
import haversine as hs
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option('display.precision', 0)
pd.options.display.float_format = "{:,.0f}".format

In [2]:
data = pd.read_csv("./data/kc_house_data.csv")
CPInts = pd.read_excel("./ScrapedData/Common_Points_of_Interest_for_King_County.xlsx")
CPInts_Xref = pd.read_excel("./ScrapedData/CodesForCommonPointsofInterest.xlsx")

In [3]:
#Merging Cross Ref Df with Points of Intereste Df
CPInts_With_Rank = CPInts.merge(CPInts_Xref, on="CODE", how="left", indicator=True)

CPInts_With_Rank['tup_lat_Long'] = list(zip(CPInts_With_Rank.Y, CPInts_With_Rank.X, CPInts_With_Rank.OBJECTID, CPInts_With_Rank.NAME,CPInts_With_Rank.PointofInterest_Rank))
data['tup_lat_Long'] = list(zip(data.lat, data.long, data.id))

In [4]:
CPInts_With_Rank_Filtered = CPInts_With_Rank.loc[CPInts_With_Rank['PointofInterest_Rank']< 4] 

In [5]:
df_Dist_From_CommonPointOfInt = pd.DataFrame(columns = ['id', 'OBJECTID', 'NAME', 'Distance', 'PointofInterest_Rank' ])
distanceConsideredClose = 7
listofValues = []
#looping Through Main mdf
for HLat, HLong, HouseParcel in data['tup_lat_Long']:
    #print(HouseParcel)
    
    #looping through df that contains interesting places 
    for Pint_Lat, PintLong, Pint_Name, pint_OBJECTID, pint_PointofInterest_Rank  in CPInts_With_Rank_Filtered['tup_lat_Long']:
        
        #getting the distance between current house and point of interest
        distance = round(hs.haversine((HLat,HLong),(Pint_Lat,PintLong), unit='mi'),2)
    
        #if point distance < 6 miles insert into a dataframe 
        if distance < distanceConsideredClose:
            values = [HouseParcel, Pint_Name, pint_OBJECTID, pint_PointofInterest_Rank, distance]
            listofValues.append(values)

keys = ['id', 'NAME', 'OBJECTID', 'PointofInterest_Rank', 'Distance']
df_Dist_From_CommonPointOfInt = pd.DataFrame(listofValues,columns = keys)


In [6]:
df_Counts_of_PointsOfInterest = df_Dist_From_CommonPointOfInt[["id", "PointofInterest_Rank"]].groupby(["id", "PointofInterest_Rank"]).size().reset_index(name='counts')

In [7]:
df_points_of_int_pivot = pd.pivot_table(df_Counts_of_PointsOfInterest, index='id', columns=['PointofInterest_Rank'], values='counts', aggfunc='max')

# Export Files To Excel

In [14]:
df_Dist_From_CommonPointOfInt.to_csv("df_CommonPointOfInt_Distance_From.csv")

In [20]:
df_Counts_of_PointsOfInterest.to_excel("df_CommonPointOfInt_Distance_From_Counts.xlsx")

In [9]:
df_points_of_int_pivot.to_excel("df_points_of_int_CrossTab.xlsx")